In [0]:
# -*- coding: utf-8 -*-
"""
Created on Mon Apr 13 22:42:56 2020

@author: Ushma
"""

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np                                
import matplotlib.pyplot as plt
from keras.datasets import cifar10

#from ResNet import 

(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
train_shape = train_images.shape

def random_resize_corp(image):
    re_size = [32,32]
    box= np.array([0,0,1,1])
    boxes = box.reshape([1,1,4])
    begin, size, bbox_for_draw = tf.image.sample_distorted_bounding_box(tf.shape(image), 
                                           bounding_boxes=boxes,
                                           aspect_ratio_range=[0.75, 1.33], 
                                           area_range=[0.08, 1.0], max_attempts=100,
                                           use_image_if_no_bounding_boxes=True)
    random_resize_img =tf.image.resize( tf.slice(image, begin, size), re_size, preserve_aspect_ratio=False)
    random_resize_img = tf.dtypes.cast(random_resize_img, dtype=tf.uint8)
    
    return random_resize_img

def color_jitter(x):
    # one can also shuffle the order of following augmentations
    # each time they are applied.
    s= 1.0
    x = tf.image.random_brightness(x, max_delta=0.8*s)
    x = tf.image.random_contrast(x, lower=1-0.8*s, upper=1+0.8*s)
    x = tf.image.random_saturation(x, lower=1-0.8*s, upper=1+0.8*s)
    x = tf.image.random_hue(x, max_delta=0.2*s)
    #x = tf.clip_by_value(x, 0, 1)
    return x

def color_drop(x):
    x = tf.image.rgb_to_grayscale(x)
    x = tf.tile(x, [1, 1, 3])
    return x

def data_augmentation_2(image):
    
    number=tf.random.uniform(shape=[],minval= 1, maxval =100)
    #print(number)
    if(number>=20):
        image= color_jitter(image)
        
    number=tf.random.uniform(shape=[],minval= 1, maxval =100)
    if(number>=80):
        image= color_drop(image)
    return image


def gaussian_blur(img, kernel_size=3):
    def gauss_kernel(channels, kernel_size, sigma):
        ax = tf.range(-kernel_size // 2 + 1.0, kernel_size // 2 + 1.0)
        xx, yy = tf.meshgrid(ax, ax)
        kernel = tf.exp(-(xx ** 2 + yy ** 2) / (2.0 * sigma ** 2))
        kernel = kernel / tf.reduce_sum(kernel)
        kernel = tf.tile(kernel[..., tf.newaxis], [1, 1, channels])
        return kernel
    #print("I am here")
    sigma = tf.random.uniform(shape=[],minval= 0.1, maxval =2.0)
    gaussian_kernel = gauss_kernel(tf.shape(img)[-1], kernel_size, sigma)
    gaussian_kernel = gaussian_kernel[..., tf.newaxis]

    return tf.nn.depthwise_conv2d(img, gaussian_kernel, [1, 1, 1, 1],padding='SAME', data_format='NHWC')



In [0]:

for i in range(len(train_images)):
    #plt.imshow(train_images[i])
    x = random_resize_corp(train_images[i])
    #plt.imshow(x)
    #print(x)
    
    x = data_augmentation_2(x)
    #plt.imshow(x)
    #print(x)
    train_images[i]=x

train_images  = tf.image.convert_image_dtype(train_images, dtype=tf.float32, saturate=False, name=None)
# gaussian_blur_images = gaussian_blur(train_images)
#plt.imshow(gaussian_blur_images[1])
print("Train Images processed")
    



Train Images processed


In [0]:
for i in range(len(test_images)):
    #plt.imshow(train_images[i])
    x = random_resize_corp(test_images[i])
    #plt.imshow(x)
    #print(x)
    
    x = data_augmentation_2(x)
    #plt.imshow(x)
    #print(x)
    test_images[i]=x

test_images  = tf.image.convert_image_dtype(test_images, dtype=tf.float32, saturate=False, name=None)
test_processed_images = gaussian_blur(test_images)
print("Test Images processed")



Test Images processed


In [0]:
#plt.imshow(test_processed_images[1])
train_dataset = (gaussian_blur_images, train_labels)
test_dataset = (test_processed_images, test_labels)

In [0]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
import tensorflow.keras as keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
import tensorflow as tf
from keras.utils import np_utils
from keras.models import load_model
from keras.datasets import cifar10
from keras.preprocessing import image
from keras import backend as K
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2

In [0]:
# New Method

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train / 255.0
x_test = x_test / 255.0

y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)
input_shape = x_train[0].shape
print(x_train.shape)
print(x_test.shape)

#plt.imshow(test_processed_images[1])
# train_dataset = (gaussian_blur_images, train_labels)
# test_dataset = (test_processed_images, test_labels)


(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [0]:
batch_size=128
out_dim=128
s= 1
temperature= 0.5
use_cosine_similarity= True
epochs= 3

In [0]:

def ResNetSimCLR(input_shape, out_dim, base_model='resnet50'):
    inputs = tf.keras.layers.Input(shape=(input_shape))

    base_encoder = tf.keras.applications.ResNet50(include_top=False, weights=None, input_tensor=None,
                                                  input_shape=None, pooling='avg')
    base_encoder.training = True
    h = base_encoder(inputs)

    # projection head
    x = tf.keras.layers.Dense(units=out_dim)(h)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.Dense(units=out_dim)(x)

    return tf.keras.Model(inputs=inputs, outputs=[h, x])


In [0]:
model = ResNetSimCLR(input_shape=input_shape, out_dim=out_dim)

criterion = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.SUM)
optimizer = tf.keras.optimizers.Adam(3e-4)

gaussian_blur_images = tf.convert_to_tensor(gaussian_blur_images, dtype=gaussian_blur_images.dtype)
train_images = train_images.repeat(epochs)

train_images = train_images.shuffle(4096)
train_images = train_images.batch(batch_size, drop_remainder=True)
train_images = train_images.prefetch(tf.data.experimental.AUTOTUNE)

AttributeError: ignored

In [0]:
def train_step(xis, xjs):
    with tf.GradientTape() as tape:
        ris, zis = model(xis)
        rjs, zjs = model(xjs)

        # normalize projection feature vectors
        zis = tf.math.l2_normalize(zis, axis=1)
        zjs = tf.math.l2_normalize(zjs, axis=1)

        # tf.summary.histogram('zis', zis, step=optimizer.iterations)
        # tf.summary.histogram('zjs', zjs, step=optimizer.iterations)

        l_pos = sim_func_dim1(zis, zjs)
        l_pos = tf.reshape(l_pos, (config['batch_size'], 1))
        l_pos /= config['temperature']
        # assert l_pos.shape == (config['batch_size'], 1), "l_pos shape not valid" + str(l_pos.shape)  # [N,1]

        negatives = tf.concat([zjs, zis], axis=0)

        loss = 0

        for positives in [zis, zjs]:
            l_neg = sim_func_dim2(positives, negatives)

            labels = tf.zeros(config['batch_size'], dtype=tf.int32)

            l_neg = tf.boolean_mask(l_neg, negative_mask)
            l_neg = tf.reshape(l_neg, (config['batch_size'], -1))
            l_neg /= config['temperature']

            # assert l_neg.shape == (
            #     config['batch_size'], 2 * (config['batch_size'] - 1)), "Shape of negatives not expected." + str(
            #     l_neg.shape)
            logits = tf.concat([l_pos, l_neg], axis=1)  # [N,K+1]
            loss += criterion(y_pred=logits, y_true=labels)

        loss = loss / (2 * config['batch_size'])
        tf.summary.scalar('loss', loss, step=optimizer.iterations)

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

In [0]:
for xis, xjs in train_dataset:
    train_step(xis, xjs)

ValueError: ignored